<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/adv_cv/class_1/9%20-%20%20Laborat%C3%B3rio/C3_W1_Lab_3_Object_Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Visão computacional avançada com TensorFlow](https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow?specialization=tensorflow-advanced-techniques) de [Laurence Moroney](https://laurencemoroney.com/) e [Andrew Ng](https://www.coursera.org/instructor/andrewng) , [DeepLearning.AI](https://www.deeplearning.ai/)

# Classificação de imagens e localização de objetos

Neste laboratório, você criará uma CNN do zero para:
- classificar o objeto principal em uma imagem
- localizá-lo desenhando caixas delimitadoras ao redor dele.

Você usará o conjunto de dados [MNIST](http://yann.lecun.com/exdb/mnist/) para sintetizar um conjunto de dados personalizado para a tarefa:
- Coloque cada imagem de "dígito" em uma tela preta de largura 75 x 75 em locais aleatórios.
- Calcule as caixas delimitadoras correspondentes para esses "dígitos".

A previsão da caixa delimitadora pode ser modelada como uma tarefa de "regressão", o que significa que o modelo preverá um valor numérico (em vez de uma categoria).

## Importações

In [ ]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np

# Testar se estamos no Google Colab
try:
  import google.colab
  IN_COLAB = True
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except:
  IN_COLAB = False
import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds

print("Tensorflow version " + tf.__version__)

# Utilitários de visualização

Essas funções são usadas para desenhar caixas delimitadoras ao redor dos dígitos.

In [ ]:
#@title Utilitários de plotagem para caixas delimitadoras [RUN ME]

im_width = 75
im_height = 75
use_normalized_coordinates = True

def draw_bounding_boxes_on_image_array(image,
                                       boxes,
                                       color=[],
                                       thickness=1,
                                       display_str_list=()):
  """Desenha caixas delimitadoras na imagem (matriz numpy).
  Args:
    image: um objeto de matriz numpy.
    boxes: uma matriz numpy bidimensional de [N, 4]: (ymin, xmin, ymax, xmax).
           As coordenadas estão no formato normalizado entre [0, 1].
    color: cor para desenhar a caixa delimitadora. O padrão é vermelho.
    thickness: espessura da linha. O valor padrão é 4.
    display_str_list_list: uma lista de cadeias de caracteres para cada caixa delimitadora.
  Raises:
    ValueError: se boxes não for uma matriz [N, 4]
  """
  image_pil = PIL.Image.fromarray(image)
  rgbimg = PIL.Image.new("RGBA", image_pil.size)
  rgbimg.paste(image_pil)
  draw_bounding_boxes_on_image(rgbimg, boxes, color, thickness,
                               display_str_list)
  return np.array(rgbimg)
  

def draw_bounding_boxes_on_image(image,
                                 boxes,
                                 color=[],
                                 thickness=1,
                                 display_str_list=()):
  """Desenha caixas delimitadoras na imagem.
  Args:
    image: um objeto PIL.Image.
    boxes: uma matriz numérica bidimensional de [N, 4]: (ymin, xmin, ymax, xmax).
           As coordenadas estão no formato normalizado entre [0, 1].
    color: cor para desenhar a caixa delimitadora. O padrão é vermelho.
    thickness: espessura da linha. O valor padrão é 4.
    display_str_list: uma lista de cadeias de caracteres para cada caixa delimitadora.
                           
  Raises:
    ValueError: se boxes não for uma matriz [N, 4]
  """
  boxes_shape = boxes.shape
  if not boxes_shape:
    return
  if len(boxes_shape) != 2 or boxes_shape[1] != 4:
    raise ValueError('A entrada deve ser do tamanho [N, 4]')
  for i in range(boxes_shape[0]):
    draw_bounding_box_on_image(image, boxes[i, 1], boxes[i, 0], boxes[i, 3],
                               boxes[i, 2], color[i], thickness, display_str_list[i])
        
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color='red',
                               thickness=1,
                               display_str=None,
                               use_normalized_coordinates=True):
  """Adiciona uma caixa delimitadora a uma imagem.
  As coordenadas da caixa delimitadora podem ser especificadas em coordenadas absolutas (pixel) ou
  coordenadas normalizadas, definindo o argumento use_normalized_coordinates.
  Args:
    image: um objeto PIL.Image.
    ymin: ymin da caixa delimitadora.
    xmin: xmin da caixa delimitadora.
    ymax: ymax da caixa delimitadora.
    xmax: xmax da caixa delimitadora.
    color: cor para desenhar a caixa delimitadora. O padrão é vermelho.
    thickness: espessura da linha. O valor padrão é 4.
    display_str_list: string a ser exibida na caixa
    use_normalized_coordinates: Se for True (padrão), tratar as coordenadas
      ymin, xmin, ymax, xmax como relativas à imagem.  Caso contrário, trate as coordenadas
      as coordenadas como absolutas.
  """
  draw = PIL.ImageDraw.Draw(image)
  im_width, im_height = image.size
  if use_normalized_coordinates:
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
  


Esses utilitários são usados para visualizar os dados e as previsões.

In [ ]:
#@title Utilitários de visualização [RUN ME]
"""
Esta célula contém funções auxiliares usadas somente para visualização
e downloads apenas. 

Você pode pular a leitura, pois há muito pouco código relacionado ao Keras ou ao Tensorflow aqui.
"""

# Configuração do Matplotlib
plt.rc('image', cmap='gray')
plt.rc('grid', linewidth=0)
plt.rc('xtick', top=False, bottom=False, labelsize='large')
plt.rc('ytick', left=False, right=False, labelsize='large')
plt.rc('axes', facecolor='F8F8F8', titlesize="large", edgecolor='white')
plt.rc('text', color='a8151a')
plt.rc('figure', facecolor='F0F0F0')# Fontes Matplotlib
MATPLOTLIB_FONT_DIR = os.path.join(os.path.dirname(plt.__file__), "mpl-data/fonts/ttf")

# extrair um lote dos conjuntos de dados. Esse código não é muito bom, ele fica muito melhor no modo eager (TODO)
def dataset_to_numpy_util(training_dataset, validation_dataset, N):
  
  # Obter um lote de cada: 10000 dígitos de validação, N dígitos de treinamento
  batch_train_ds = training_dataset.unbatch().batch(N)
  
  # Execução ansiosa: percorre os conjuntos de dados normalmente
  if tf.executing_eagerly():
    for validation_digits, (validation_labels, validation_bboxes) in validation_dataset:
      validation_digits = validation_digits.numpy()
      validation_labels = validation_labels.numpy()
      validation_bboxes = validation_bboxes.numpy()
      break
    for training_digits, (training_labels, training_bboxes) in batch_train_ds:
      training_digits = training_digits.numpy()
      training_labels = training_labels.numpy()
      training_bboxes = training_bboxes.numpy()
      break
  
  # esses foram codificados na forma one-hot no conjunto de dados
  validation_labels = np.argmax(validation_labels, axis=1)
  training_labels = np.argmax(training_labels, axis=1)
  
  return (training_digits, training_labels, training_bboxes,
          validation_digits, validation_labels, validation_bboxes)

# criar dígitos de fontes locais para teste
def create_digits_from_local_fonts(n):
  font_labels = []
  img = PIL.Image.new('LA', (75*n, 75), color = (0,255)) # Formato 'LA': preto no canal 0, alfa no canal 1
  font1 = PIL.ImageFont.truetype(os.path.join(MATPLOTLIB_FONT_DIR, 'DejaVuSansMono-Oblique.ttf'), 25)
  font2 = PIL.ImageFont.truetype(os.path.join(MATPLOTLIB_FONT_DIR, 'STIXGeneral.ttf'), 25)
  d = PIL.ImageDraw.Draw(img)
  for i in range(n):
    font_labels.append(i%10)
    d.text((7+i*75,0 if i<10 else -4), str(i%10), fill=(255,255), font=font1 if i<10 else font2)
  font_digits = np.array(img.getdata(), np.float32)[:,0] / 255.0 # preto no canal 0, alfa no canal 1 (descartado)
  font_digits = np.reshape(np.stack(np.split(np.reshape(font_digits, [75, 75*n]), n, axis=1), axis=0), [n, 75*75])
  return font_digits, font_labels

# Utilitário para exibir uma linha de dígitos com suas previsões
def display_digits_with_boxes(digits, predictions, labels, pred_bboxes, bboxes, iou, title):

  n = 10

  indexes = np.random.choice(len(predictions), size=n)
  n_digits = digits[indexes]
  n_predictions = predictions[indexes]
  n_labels = labels[indexes]

  n_iou = []
  if len(iou) > 0:
    n_iou = iou[indexes]

  if (len(pred_bboxes) > 0):
    n_pred_bboxes = pred_bboxes[indexes,:]

  if (len(bboxes) > 0):
    n_bboxes = bboxes[indexes,:]


  n_digits = n_digits * 255.0
  n_digits = n_digits.reshape(n, 75, 75)
  fig = plt.figure(figsize=(20, 4))
  plt.title(title)
  plt.yticks([])
  plt.xticks([])
  
  for i in range(10):
    ax = fig.add_subplot(1, 10, i+1)
    bboxes_to_plot = []
    if (len(pred_bboxes) > i):
      bboxes_to_plot.append(n_pred_bboxes[i])
    
    if (len(bboxes) > i):
      bboxes_to_plot.append(n_bboxes[i])

    img_to_draw = draw_bounding_boxes_on_image_array(image=n_digits[i], boxes=np.asarray(bboxes_to_plot), color=['red', 'green'], display_str_list=["true", "pred"])
    plt.xlabel(n_predictions[i])
    plt.xticks([])
    plt.yticks([])
    
    if n_predictions[i] != n_labels[i]:
      ax.xaxis.label.set_color('red')

    
    
    plt.imshow(img_to_draw)

    if len(iou) > i :
      color = "black"
      if (n_iou[i][0] < iou_threshold):
        color = "red"
      ax.text(0.2, -0.3, "iou: %s" %(n_iou[i][0]), color=color, transform=ax.transAxes)


# Utilitário para exibir curvas de treinamento e validação
def plot_metrics(metric_name, title, ylim=5):
  plt.title(title)
  plt.ylim(0,ylim)
  plt.plot(history.history[metric_name],color='blue',label=metric_name)
  plt.plot(history.history['val_' + metric_name],color='green',label='val_' + metric_name)

## Seleção de estratégias

### Detecção de TPU ou GPU

Dependendo do hardware disponível, você usará diferentes estratégias de distribuição.  Para uma conhecer as estratégias de distribuição de processamento, consulte o curso ["Custom and Distributed Training with TensorFlow"](https://www.coursera.org/learn/custom-distributed-training-with-tensorflow), semana 4, "_Distributed Training_".

- Se a TPU estiver disponível, você usará a estratégia TPU.
Caso contrário:
- Se houver mais de uma GPU disponível, você usará a estratégia Mirrored
- Se uma GPU estiver disponível ou se apenas a CPU estiver disponível, você usará a estratégia padrão.

In [ ]:
# Detectar hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Selecionar a estratégia de distribuição apropriada
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu) # A ida e volta entre a TPU e o host é cara. É melhor executar 128 lotes na TPU antes de reportar.
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # Estratégia padrão que funciona na CPU e em uma única GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # Estratégia padrão que funciona na CPU e em uma única GPU
  print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

### Parâmetros

O tamanho do lote global é o tamanho do lote por réplica (64 neste caso) vezes o número de réplicas na estratégia de distribuição.

In [ ]:
BATCH_SIZE = 64 * strategy.num_replicas_in_sync  # Tamanho do lote global.
# O tamanho do lote global será automaticamente fragmentado em todas as
# réplicas pela API tf.data.Dataset. Uma única TPU tem 8 núcleos.
# A prática recomendada é dimensionar o tamanho do lote de acordo com o número de
# réplicas (núcleos). A taxa de aprendizado também deve ser aumentada.

## Carregando e pré-processando o conjunto de dados

Defina algumas funções auxiliares que farão o pré-processamento dos dados:
- `read_image_tfds`: sobrepõe aleatoriamente a imagem do "dígito" em uma tela maior.
- `get_training_dataset`: carrega os dados e os divide para obter o conjunto de treinamento.
- `get_validation_dataset`: carrega e divide os dados para obter o conjunto de validação.

In [ ]:
'''
Transforma cada imagem do conjunto de dados, colando-a em uma tela de 75x75 em locais aleatórios.
'''
def read_image_tfds(image, label):
    xmin = tf.random.uniform((), 0 , 48, dtype=tf.int32)
    ymin = tf.random.uniform((), 0 , 48, dtype=tf.int32)
    image = tf.reshape(image, (28,28,1,))
    image = tf.image.pad_to_bounding_box(image, ymin, xmin, 75, 75)
    image = tf.cast(image, tf.float32)/255.0
    xmin = tf.cast(xmin, tf.float32)
    ymin = tf.cast(ymin, tf.float32)
   
    xmax = (xmin + 28) / 75
    ymax = (ymin + 28) / 75
    xmin = xmin / 75
    ymin = ymin / 75
    return image, (tf.one_hot(label, 10), [xmin, ymin, xmax, ymax])
  
'''
Carrega e mapeia a divisão de treinamento do conjunto de dados usando a função map.
Observe que tentamos carregar a versão gcs, pois a TPU só pode trabalhar com conjuntos de dados no Google Cloud Storage.
'''
def get_training_dataset():
      
      with  strategy.scope():
        dataset = tfds.load("mnist", split="train", as_supervised=True, try_gcs=True)
        dataset = dataset.map(read_image_tfds, num_parallel_calls=16)
        dataset = dataset.shuffle(5000, reshuffle_each_iteration=True)
        dataset = dataset.repeat() # Obrigatório para o Keras por enquanto
        dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) # drop_remainder é importante na TPU, o tamanho do lote deve ser fixo
        dataset = dataset.prefetch(-1)  # Busque os próximos lotes enquanto treina no lote atual (-1: ajuste automático do tamanho do buffer de pré-busca)
      return dataset

'''
Carrega e mapeia a divisão de validação do conjunto de dados usando a função map.
Observe que tentamos carregar a versão gcs, pois a TPU só pode trabalhar com conjuntos de dados no Google Cloud Storage.
''' 
def get_validation_dataset():
    dataset = tfds.load("mnist", split="test", as_supervised=True, try_gcs=True)
    dataset = dataset.map(read_image_tfds, num_parallel_calls=16)

    #dataset = dataset.cache() esse pequeno conjunto de dados pode ser totalmente armazenado em cache na RAM
    dataset = dataset.batch(10000, drop_remainder=True) # 10000 itens no conjunto de dados de avaliação, todos em um lote
    dataset = dataset.repeat() # Obrigatório para o Keras por enquanto
    return dataset

# instanciar os conjuntos de dados
with strategy.scope():
  training_dataset = get_training_dataset()
  validation_dataset = get_validation_dataset()

### Visualizar dados

In [ ]:
(training_digits, training_labels, training_bboxes,
 validation_digits, validation_labels, validation_bboxes) = dataset_to_numpy_util(training_dataset, validation_dataset, 10)

display_digits_with_boxes(training_digits, training_labels, training_labels, np.array([]), training_bboxes, np.array([]), "dígitos de treinamento e seus rótulos")
display_digits_with_boxes(validation_digits, validation_labels, validation_labels, np.array([]), validation_bboxes, np.array([]), "dígitos de validação e seus rótulos")


## Definir a rede

Aqui, você definirá sua CNN personalizada. 
- `feature_extractor`: essas camadas convolucionais extraem as características da imagem.
- `classifier`:  define a camada de saída que prevê entre 10 categorias (dígitos de 0 a 9)
- `bounding_box_regression`: define a camada de saída que prevê 4 valores numéricos, que definem as coordenadas da caixa delimitadora (xmin, ymin, xmax, ymax)
- `final_model`: Combina as camadas de extração de recursos, classificação e previsão de caixa delimitadora.  
  - Observe que esse é outro exemplo de um modelo de ramificação, pois o modelo se divide para produzir dois tipos de saída (uma categoria e um conjunto de números).  
  - Como você aprendeu a usar a API funcional anteriormente, você tem a flexibilidade de definir esse tipo de modelo de ramificação!
- `define_and_compile_model`: escolha o otimizador e as métricas e, em seguida, compile o modelo.

In [ ]:
'''
O extrator de características é a CNN, que é composta de camadas de convolução e agrupamento(pooling).
'''
def feature_extractor(inputs):
    x = tf.keras.layers.Conv2D(16, activation='relu', kernel_size=3, input_shape=(75, 75, 1))(inputs)
    x = tf.keras.layers.AveragePooling2D((2, 2))(x)

    x = tf.keras.layers.Conv2D(32,kernel_size=3,activation='relu')(x)
    x = tf.keras.layers.AveragePooling2D((2, 2))(x)

    x = tf.keras.layers.Conv2D(64,kernel_size=3,activation='relu')(x)
    x = tf.keras.layers.AveragePooling2D((2, 2))(x)

    return x

'''
dense_layers adiciona uma camada achatada e densa.
Isso seguirá as camadas de extração de recursos
'''
def dense_layers(inputs):
  x = tf.keras.layers.Flatten()(inputs)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  return x


'''
O classificador define a saída da classificação.
Ele tem um conjunto de camadas totalmente conectadas e uma camada softmax.
'''
def classifier(inputs):

  classification_output = tf.keras.layers.Dense(10, activation='softmax', name = 'classification')(inputs)
  return classification_output


'''
Essa função define o resultado da regressão para a previsão da caixa delimitadora. 
Observe que temos quatro saídas correspondentes a (xmin, ymin, xmax, ymax)
'''
def bounding_box_regression(inputs):
    bounding_box_regression_output = tf.keras.layers.Dense(units = '4', name = 'bounding_box')(inputs)
    return bounding_box_regression_output


def final_model(inputs):
    feature_cnn = feature_extractor(inputs)
    dense_output = dense_layers(feature_cnn)

    '''
    O modelo se ramifica aqui.  
    A saída da camada densa é alimentada em duas ramificações:
    classification_output e bounding_box_output
    '''
    classification_output = classifier(dense_output)
    bounding_box_output = bounding_box_regression(dense_output)

    model = tf.keras.Model(inputs = inputs, outputs = [classification_output, bounding_box_output])

    return model
  

def define_and_compile_model(inputs):
  model = final_model(inputs)
  
  model.compile(optimizer='adam', 
              loss = {'classification' : 'categorical_crossentropy',
                      'bounding_box' : 'mse'
                     },
              metrics = {'classification' : 'accuracy',
                         'bounding_box' : 'mse'
                        })
  return model

    
with strategy.scope():
  inputs = tf.keras.layers.Input(shape=(75, 75, 1,))
  model = define_and_compile_model(inputs)

# imprimir camadas do modelo
model.summary()

### Treinar e validar o modelo

Treine o modelo.  
- Você pode escolher o número de épocas, dependendo do nível de desempenho que deseja e do tempo de que dispõe.
- Cada época levará apenas alguns segundos se você estiver usando a TPU.

In [ ]:
EPOCHS = 10 # 45
steps_per_epoch = 60000//BATCH_SIZE  # 60.000 itens nesse conjunto de dados
validation_steps = 1

history = model.fit(training_dataset,
                    steps_per_epoch=steps_per_epoch, validation_data=validation_dataset, validation_steps=validation_steps, epochs=EPOCHS)

loss, classification_loss, bounding_box_loss, classification_accuracy, bounding_box_mse = model.evaluate(validation_dataset, steps=1)
print("Validation accuracy: ", classification_accuracy)

In [ ]:
plot_metrics("classification_loss", "Perda de Classificação")

In [ ]:
plot_metrics("bounding_box_loss", "Perda de Bounding Box")

## Intersecção sobre união

Calcule a métrica I-O-U para avaliar o desempenho do modelo.

In [ ]:
def intersection_over_union(pred_box, true_box):
    xmin_pred, ymin_pred, xmax_pred, ymax_pred =  np.split(pred_box, 4, axis = 1)
    xmin_true, ymin_true, xmax_true, ymax_true = np.split(true_box, 4, axis = 1)

    smoothing_factor = 1e-10

    xmin_overlap = np.maximum(xmin_pred, xmin_true)
    xmax_overlap = np.minimum(xmax_pred, xmax_true)
    ymin_overlap = np.maximum(ymin_pred, ymin_true)
    ymax_overlap = np.minimum(ymax_pred, ymax_true)

    pred_box_area = (xmax_pred - xmin_pred) * (ymax_pred - ymin_pred)
    true_box_area = (xmax_true - xmin_true) * (ymax_true - ymin_true)

    overlap_area = np.maximum((xmax_overlap - xmin_overlap), 0)  * np.maximum((ymax_overlap - ymin_overlap), 0)
    union_area = (pred_box_area + true_box_area) - overlap_area
    
    iou = (overlap_area + smoothing_factor) / (union_area + smoothing_factor)

    return iou

### Visualizar previsões
O código a seguir fará previsões e visualizará a classificação e as caixas delimitadoras previstas.
- Os rótulos das caixas delimitadoras verdadeiras estarão em verde, e as caixas delimitadoras previstas pelo modelo estarão em vermelho.
- O número previsto é mostrado abaixo da imagem.

In [ ]:
# reconhecer os dígitos de validação
predictions = model.predict(validation_digits, batch_size=64)
predicted_labels = np.argmax(predictions[0], axis=1)

predicted_bboxes = predictions[1]

iou = intersection_over_union(predicted_bboxes, validation_bboxes)

iou_threshold = 0.6

print("Número de previsões em que iou > limite(%s): %s" % (iou_threshold, (iou >= iou_threshold).sum()))
print("Número de previsões em que iou < limite(%s): %s" % (iou_threshold, (iou < iou_threshold).sum()))


display_digits_with_boxes(validation_digits, predicted_labels, validation_labels, predicted_bboxes, validation_bboxes, iou, "Valores reais e previstos")